In [89]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn import model_selection
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [90]:
path = "C:\\Users\\Ivand\\Repos\\banks\\data"

In [107]:
df = pd.read_csv(path + "\\BanksDFUsable.csv")
df = df.drop(df.columns[0], axis = 1)
df_ratings = pd.read_csv(path + "\\RatingsClean.csv")
df_normatives = pd.read_csv(path + "\\NormativesUsable.csv")
df_normatives.drop(df_normatives.columns[0], axis = 1, inplace = True)

In [108]:
df = df[(df.Year >= 2015)&(df.Year <= 2020)]

In [109]:
#count non-zeros in each column
df.astype(bool).sum(axis = 0).sort_values()

BadBorrowing_Me               1316
Revaluation_CBR               2796
DefaultIn365Days              3817
CBRCredits_CBR                6090
DerivativeLiabilities_CBR     6105
                             ...  
RetainedEarnings_CBR         40880
Month                        40882
Year                         40882
Date                         40882
REGN                         40882
Length: 78, dtype: int64

In [110]:
df["BorrowShort_Me"]

96       -568957281.0
97       -619951278.0
98       -559407686.0
99       -528555250.0
100      -481856127.0
             ...     
129558      -606650.0
129559      -484302.0
129560      -401890.0
129561      -399043.0
129562      -308455.0
Name: BorrowShort_Me, Length: 40882, dtype: float64

In [111]:
df_ratings = df_ratings.rename(columns = {"regn" : "REGN"})

In [112]:
df.REGN = df.REGN.apply(str)
df_ratings.REGN = df_ratings.REGN.apply(str)

In [113]:
df_ratings[df_ratings.REGN == "1"]

,Moodys,name,ExpertRA,short_name,form,REGN,region,ownership,personal_page,app,Year,Month,foreign
31345,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,1,1.0
31346,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,2,1.0
31347,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,3,1.0
31348,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,4,1.0
31349,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31412,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,8,1.0
31413,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,9,1.0
31414,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,10,1.0
31415,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,11,1.0


In [114]:
df_normatives.REGN

0           1
1           1
2           1
3           1
4           1
         ... 
37760    3538
37761    3538
37762    3538
37763    3538
37764    3538
Name: REGN, Length: 37765, dtype: int64

In [115]:
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-К", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Д", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-С", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Г", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Р", "", x))
df_normatives.REGN = df_normatives.REGN.apply(str)

df_full = df.merge(df_ratings, on = ["Year", "Month", "REGN"], how = "left")
#этот фильтр удаляет только национальный клиринговый центр
df_full = df_full[df_full.short_name == df_full.short_name]
df_full = df_full.merge(df_normatives, on = ["REGN", "Year", "Month"], how = "left")


In [116]:
df_full["BorrowShort_Me"]

0       -568957281.0
1       -619951278.0
2       -559407686.0
3       -528555250.0
4       -481856127.0
            ...     
40885      -606650.0
40886      -484302.0
40887      -401890.0
40888      -399043.0
40889      -308455.0
Name: BorrowShort_Me, Length: 40890, dtype: float64

In [117]:
df_full = df_full.fillna(0)

In [118]:
#df_full.to_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\BanksDFFull.csv")

In [119]:
#list(df_full.columns)

In [120]:
df_full = df_full[df_full.Assets != 0]
df_full = df_full[df_full.NetIncome != 0]


In [121]:
df_full["BorrowShort_Me"]

0       -568957281.0
1       -619951278.0
2       -559407686.0
3       -528555250.0
4       -481856127.0
            ...     
40885      -606650.0
40886      -484302.0
40887      -401890.0
40888      -399043.0
40889      -308455.0
Name: BorrowShort_Me, Length: 40885, dtype: float64

In [122]:
np.array(df_full.columns)

array(['REGN', 'Year', 'Month', 'NetIncome', 'NetSecurityIncome',
       'NetDerivativeIncome', 'NetFeeIncome', 'NetInvestmentIncome',
       'NetInterestIncome', 'Assets', 'Liabilities', 'Equity',
       'RequiredReserves', 'Securities', 'Investment', 'RetainedEarning',
       'DefaultIn365Days', 'DefaultIn730Days', 'DefaultIn10000Days',
       'ImmobilizedAssets', 'Reserves_Me', 'LendCorporate_Me',
       'LendRetail', 'BorrowCorp_Me', 'BorrowRetail_Me',
       'LendInterbank_Me', 'BorrowInterbank_Me', 'LendShort_Me',
       'BorrowShort_Me', 'LendLong_Me', 'BorrowLong_Me', 'Cash_Me',
       'BondsIssued_Me', 'BadCredits_Me', 'LendState_Me',
       'BorrowState_Me', 'CurrentAccounts_Me', 'TradingAccounts_Me',
       'BadBorrowing_Me', 'ReservesCorrected_Me', 'Deposits_Me',
       'Credits_Me', 'IncomeRetail_Me', 'IncomeCorporates_Me',
       'ExpenditureRetail_Me', 'ExpenditureCorporates_Me',
       'ExpendituresReserves_Me', 'DepositsRetail_Me',
       'DepositsCorporates_Me', 'Cred

In [123]:
df_full["CreditsReturn"] = df_full["CreditsIncome"]/df_full["Credits_Me"]
df_full["DepositsInterest"] = df_full["DepositsExpenditure"]/df_full["Deposits_Me"]

df_full["CreditsReturnCorporates"] = df_full["IncomeCorporates_Me"]/df_full["CreditsCorporates_Me"]
df_full["DepositsInterestCorporates"] = df_full["ExpenditureCorporates_Me"]/df_full["DepositsCorporates_Me"]

df_full["CreditsReturnRetail"] = df_full["IncomeRetail_Me"]/df_full["CreditsRetail_Me"]
df_full["DepositsInterestRetail"] = df_full["ExpenditureRetail_Me"]/df_full["DepositsRetail_Me"]

df_full["CostOfRisk"] = df_full["ExpendituresReserves_Me"]/(df_full["Credits_Me"] + df_full["ReservesCorrected_Me"])

df_full["CorporateReturnShare"] = df_full["CreditsReturnCorporates"]/df_full["CreditsReturn"]
df_full["CorporateExpenditureShare"] = df_full["DepositsInterestCorporates"]/df_full["DepositsInterest"]

df_full["InterbankShareInLiabs"] = df_full["BorrowInterbank_Me"]/df_full["Liabilities"]

In [124]:
df_full["ROE"] = df_full.NetIncome/df_full.Equity
df_full["ROA"] = df_full.NetIncome/df_full.Assets
df_full["ReservesChange"] = (df_full["RequiredReserves"]/df_full.groupby("REGN")["RequiredReserves"].shift(1)).fillna(0)
df_full["GeneralLiquidity"] = df_full["LendShort_Me"]/df_full["BorrowShort_Me"]
df_full["EquityToImmobilizedAssets"] = df_full["LendShort_Me"]/df_full["BorrowShort_Me"]


divide_by_net_income = ["NetSecurityIncome", 'NetDerivativeIncome',
         'NetFeeIncome', 'NetInvestmentIncome',
         'NetInterestIncome']
df_full.loc[:, divide_by_net_income] = df_full.loc[:, divide_by_net_income].div(df_full["NetIncome"], axis=0)

divide_by_assets = ["NetIncome", 'Liabilities', 'Equity', 'Securities',
       'Investment', 'RetainedEarning','Reserves_Me', 'LendCorporate_Me', 'LendRetail',
       'BorrowCorp_Me', 'BorrowRetail_Me', 'LendInterbank_Me',
       'BorrowInterbank_Me', 'LendShort_Me', 'BorrowShort_Me', 'LendLong_Me',
       'BorrowLong_Me', 'Cash_Me', 'BondsIssued_Me', 'BadCredits_Me',
       'LendState_Me', 'BorrowState_Me', 'CurrentAccounts_Me',
       'TradingAccounts_Me', 'BadBorrowing_Me', 'Unclassified',
       'CBRDeposits_CBR', 'ObligatoryReserves_CBR', 'CreditsToBanks_CBR',
       'Securities_CBR', 'ShareCapitalParticipation_CBR',
       'CreditPortfolio_CBR', 'ProfitableDerivatives_CBR', 'CoreFunds_CBR',
       'Intangibles_CBR', 'DeferredTaxAsset_CBR', 'OtherAssets_CBR',
       'CBRCredits_CBR', 'BanksFunds_CBR', 'ClientFunds_CBR',
       'SecuritiesIssued_CBR', 'DerivativeLiabilities_CBR',
       'OtherLiabilities_CBR', 'Revaluation_CBR', 'MainEquity_CBR',
       'ExtraEquity_CBR', 'ReserveFund_CBR', 'SecuritiesRevaluation_CBR',
       'RetainedEarnings_CBR', 'LiquidAssets_norm',
       'RiskFree_norm', 'RiskWeightedI0_norm', 'RiskWeightedIV0_norm','RiskWeightedII0_norm', 'LiquidityMonth_norm',
       'LiabilitiesOnDemandT_norm', 'HighlyLiquidAssets_norm','RiskyOperations0_norm',
       'RiskyOperationsII_norm', 'LiquidityDay_norm',
       'LiabilitiesOnDemandM_norm',"ShortCreditAssets_norm", 'RiskyOperationsI_norm', 'CreditLiabilities_norm','MarketRisk0_norm',
       'InterestRisk0_norm', 'MarketRiskII_norm', 'InterestRiskII_norm']
df_full.loc[:, divide_by_assets] = df_full.loc[:, divide_by_assets].div(df_full["Assets"], axis=0)

In [125]:
factors_list = ['REGN', 'Year', 'Month', 'ownership', 'personal_page', 'app', 'foreign']

In [126]:
df_full.drop(["Date", "short_name", "name"], axis = 1, inplace = True)

In [127]:
#we can see that the least populated class in Moodys (Ca) has only three features. Lets group all the Ca*
#classes to one class "C"
df_full.groupby(df_full.Moodys).count()

,REGN,Year,Month,NetIncome,NetSecurityIncome,NetDerivativeIncome,NetFeeIncome,NetInvestmentIncome,NetInterestIncome,Assets,...,DepositsInterestRetail,CostOfRisk,CorporateReturnShare,CorporateExpenditureShare,InterbankShareInLiabs,ROE,ROA,ReservesChange,GeneralLiquidity,EquityToImmobilizedAssets
Moodys,,,,,,,,,,,,,,,,,,,,,
B1,430,430,430,430,430,430,430,430,430,430,...,430,430,430,430,430,430,430,430,430,430
B2,665,665,665,665,665,665,665,665,665,665,...,665,665,665,665,665,665,665,665,665,665
B3,700,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700
Ba1,452,452,452,452,452,452,452,452,452,452,...,452,452,416,452,452,452,452,452,452,452
Ba2,392,392,392,392,392,392,392,392,392,392,...,382,392,392,374,392,392,392,392,392,392
Ba3,421,421,421,421,421,421,421,421,421,421,...,419,421,421,419,421,421,421,421,421,421
Baa3,130,130,130,130,130,130,130,130,130,130,...,130,130,130,130,130,130,130,130,130,130
Ca,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Caa1,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99


In [128]:
df_full.loc[(df_full.Moodys == "Ca") | \
             (df_full.Moodys == "Caa1") | \
             (df_full.Moodys == "Caa2") | \
             (df_full.Moodys == "Caa3"), "Moodys"] = "C"

In [129]:
#with ExpertRA the situation is even more critical; the number of discrete classes is extremely large. Lets 
#group them in bigger classes
df_full.groupby(df_full.ExpertRA).count()

,REGN,Year,Month,NetIncome,NetSecurityIncome,NetDerivativeIncome,NetFeeIncome,NetInvestmentIncome,NetInterestIncome,Assets,...,DepositsInterestRetail,CostOfRisk,CorporateReturnShare,CorporateExpenditureShare,InterbankShareInLiabs,ROE,ROA,ReservesChange,GeneralLiquidity,EquityToImmobilizedAssets
ExpertRA,,,,,,,,,,,,,,,,,,,,,
A (I),511,511,511,511,511,511,511,511,511,511,...,511,511,502,511,511,511,511,511,511,511
A (II),469,469,469,469,469,469,469,469,469,469,...,469,469,469,469,469,469,469,469,469,469
A (III),662,662,662,662,662,662,662,662,662,662,...,662,662,662,662,662,662,662,662,662,662
A+,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
A+ (I),175,175,175,175,175,175,175,175,175,175,...,175,175,164,164,175,175,175,175,175,175
A+ (II),117,117,117,117,117,117,117,117,117,117,...,117,117,117,107,117,117,117,117,117,117
A+ (III),291,291,291,291,291,291,291,291,291,291,...,291,291,291,291,291,291,291,291,291,291
A++,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159
B,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49


In [130]:
df_full.loc[(df_full.ExpertRA == "ruC") | \
             (df_full.ExpertRA == "ruCC") | \
             (df_full.ExpertRA == "ruCCC"), "ExpertRA"] = "ruC"
df_full.loc[(df_full.ExpertRA == "A+"), "ExpertRA"] = "A+ (I)"
df_full.loc[(df_full.ExpertRA == "A (I)") | \
             (df_full.ExpertRA == "A (II)") | \
             (df_full.ExpertRA == "A (III)"), "ExpertRA"] = "A"
df_full.loc[(df_full.ExpertRA == "A+ (I)") | \
             (df_full.ExpertRA == "A+ (II)") | \
             (df_full.ExpertRA == "A+ (III)"), "ExpertRA"] = "A+"
df_full.loc[(df_full.ExpertRA == "A++ (I)") | \
             (df_full.ExpertRA == "A++ (II)") | \
             (df_full.ExpertRA == "A++ (III)"), "ExpertRA"] = "A++"
df_full.loc[(df_full.ExpertRA == "С") | \
             (df_full.ExpertRA == "C+") | \
             (df_full.ExpertRA == "C++") | \
             (df_full.ExpertRA == "E"), "ExpertRA"] = "C"
df_full.loc[(df_full.ExpertRA == "ruAA") | \
             (df_full.ExpertRA == "ruAA+") | \
             (df_full.ExpertRA == "ruAA-"), "ExpertRA"] = "ruAA"
df_full.loc[(df_full.ExpertRA == "B") | \
             (df_full.ExpertRA == "B+") | \
             (df_full.ExpertRA == "B++"), "ExpertRA"] = "B"
df_full.loc[(df_full.ExpertRA == "A") | \
             (df_full.ExpertRA == "A+") | \
             (df_full.ExpertRA == "A++"), "ExpertRA"] = "A"
df_full.loc[(df_full.ExpertRA == "ruA") | \
             (df_full.ExpertRA == "ruA+") | \
             (df_full.ExpertRA == "ruA-"), "ExpertRA"] = "ruA"
df_full.loc[(df_full.ExpertRA == "ruB") | \
             (df_full.ExpertRA == "ruB+") | \
             (df_full.ExpertRA == "ruB-"), "ExpertRA"] = "ruB"
df_full.loc[(df_full.ExpertRA == "ruBB") | \
             (df_full.ExpertRA == "ruBB+") | \
             (df_full.ExpertRA == "ruBB-"), "ExpertRA"] = "ruBB"
df_full.loc[(df_full.ExpertRA == "ruBBB") | \
             (df_full.ExpertRA == "ruBBB+") | \
             (df_full.ExpertRA == "ruBBB-"), "ExpertRA"] = "ruBBB"

In [131]:
df_full = pd.get_dummies(df_full, columns = ["ownership", "form", "region"], drop_first = True)

In [132]:
df_with_Moodys = df_full[df_full.Moodys != "no"]
df_with_Expertra = df_full[df_full.ExpertRA != "no"]

In [133]:
df_full.columns

Index(['REGN', 'Year', 'Month', 'NetIncome', 'NetSecurityIncome',
       'NetDerivativeIncome', 'NetFeeIncome', 'NetInvestmentIncome',
       'NetInterestIncome', 'Assets',
       ...
       'CorporateExpenditureShare', 'InterbankShareInLiabs', 'ROE', 'ROA',
       'ReservesChange', 'GeneralLiquidity', 'EquityToImmobilizedAssets',
       'ownership_state', 'form_public', 'region_notmoscow'],
      dtype='object', length=126)

In [83]:
seed = 42
X_Moodys = df_with_Moodys.drop(["Moodys", "ExpertRA"], axis = 1)
X_Expertra = df_with_Expertra.drop(["Moodys", "ExpertRA"], axis = 1)

Moodys = df_with_Moodys["Moodys"]
Expertra = df_with_Expertra["ExpertRA"]

X_Moodys_train, X_Moodys_test, Moodys_train, Moodys_test =\
                    model_selection.train_test_split(X_Moodys, Moodys, test_size=0.2, random_state=seed)
X_Expertra_train, X_Expertra_test, Expertra_train, Expertra_test =\
                    model_selection.train_test_split(X_Expertra, Expertra, test_size=0.2, random_state=seed)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
Moodys_train.reset_index(inplace=True, drop=True)
Moodys_test.reset_index(inplace=True, drop=True)
Expertra_train.reset_index(inplace=True, drop=True)
Expertra_test.reset_index(inplace=True, drop=True)

NameError: name 'X_train' is not defined

In [135]:
list(df_full.columns)

['REGN',
 'Year',
 'Month',
 'NetIncome',
 'NetSecurityIncome',
 'NetDerivativeIncome',
 'NetFeeIncome',
 'NetInvestmentIncome',
 'NetInterestIncome',
 'Assets',
 'Liabilities',
 'Equity',
 'RequiredReserves',
 'Securities',
 'Investment',
 'RetainedEarning',
 'DefaultIn365Days',
 'DefaultIn730Days',
 'DefaultIn10000Days',
 'ImmobilizedAssets',
 'Reserves_Me',
 'LendCorporate_Me',
 'LendRetail',
 'BorrowCorp_Me',
 'BorrowRetail_Me',
 'LendInterbank_Me',
 'BorrowInterbank_Me',
 'LendShort_Me',
 'BorrowShort_Me',
 'LendLong_Me',
 'BorrowLong_Me',
 'Cash_Me',
 'BondsIssued_Me',
 'BadCredits_Me',
 'LendState_Me',
 'BorrowState_Me',
 'CurrentAccounts_Me',
 'TradingAccounts_Me',
 'BadBorrowing_Me',
 'ReservesCorrected_Me',
 'Deposits_Me',
 'Credits_Me',
 'IncomeRetail_Me',
 'IncomeCorporates_Me',
 'ExpenditureRetail_Me',
 'ExpenditureCorporates_Me',
 'ExpendituresReserves_Me',
 'DepositsRetail_Me',
 'DepositsCorporates_Me',
 'CreditsCorporates_Me',
 'CreditsRetail_Me',
 'Unclassified',
 'CBR

In [139]:
df_for_share = df_full[['REGN', 'Year', 'Month', 'NetIncome', 'NetSecurityIncome',
       'NetDerivativeIncome', 'NetFeeIncome', 'NetInvestmentIncome',
       'NetInterestIncome', 'Assets', 'Liabilities', 'Equity',
       'RequiredReserves', 'Securities', 'Investment', 'RetainedEarning',
       'ImmobilizedAssets', 'Reserves_Me', 'LendCorporate_Me', 'LendRetail',
       'BorrowCorp_Me', 'BorrowRetail_Me', 'LendInterbank_Me',
       'BorrowInterbank_Me', 'LendShort_Me', 'BorrowShort_Me', 'LendLong_Me',
       'BorrowLong_Me', 'Cash_Me', 'BondsIssued_Me', 'BadCredits_Me',
       'LendState_Me', 'BorrowState_Me', 'CurrentAccounts_Me',
       'TradingAccounts_Me', 'BadBorrowing_Me', 'ReservesCorrected_Me',
       'Unclassified', 'CBRDeposits_CBR', 'ObligatoryReserves_CBR',
       'CreditsToBanks_CBR', 'Securities_CBR', 'ShareCapitalParticipation_CBR',
       'CreditPortfolio_CBR', 'ProfitableDerivatives_CBR', 'CoreFunds_CBR',
       'Intangibles_CBR', 'DeferredTaxAsset_CBR', 'OtherAssets_CBR',
       'CBRCredits_CBR', 'BanksFunds_CBR', 'ClientFunds_CBR',
       'SecuritiesIssued_CBR', 'DerivativeLiabilities_CBR',
       'OtherLiabilities_CBR', 'Revaluation_CBR', 'MainEquity_CBR',
       'ExtraEquity_CBR', 'ReserveFund_CBR', 'SecuritiesRevaluation_CBR',
       'RetainedEarnings_CBR', 'personal_page', 'app',
       'foreign','LiquidAssets_norm', 'RiskFree_norm','CapitalAdequacy_norm',
       'LiquidityMonth_norm','LiquidityDay_norm',
       'LiabilitiesOnDemandM_norm', 
       'LiquidityLong_norm', 'CreditsToInsidersCoefficient_norm',  'ROE',
       'ROA', 'ReservesChange', 'GeneralLiquidity',
       'EquityToImmobilizedAssets',  
       'CreditsIncome',
       'DepositsExpenditure',
       'Deposits_Me',
       'Credits_Me',
       'IncomeRetail_Me',
       'IncomeCorporates_Me',
       'ExpenditureRetail_Me',
       'ExpenditureCorporates_Me',
       'ExpendituresReserves_Me',
       'DepositsRetail_Me',
       'DepositsCorporates_Me',
       'CreditsCorporates_Me',
       'CreditsReturn',
       'DepositsInterest',
       'CreditsReturnCorporates',
       'DepositsInterestCorporates',
       'CreditsReturnRetail',
       'DepositsInterestRetail',
       'CostOfRisk',
       'CorporateReturnShare',
       'CorporateExpenditureShare',
       'InterbankShareInLiabs',
       'CreditsRetail_Me','ownership_state', 'form_public',
       'region_notmoscow', "Moodys", "ExpertRA", "DefaultIn365Days", "DefaultIn730Days","DefaultIn10000Days"]]

In [137]:
df_for_share.EquityToImmobilizedAssets

0       -0.482442
1       -0.526836
2       -0.470002
3       -0.508174
4       -0.410839
           ...   
40885   -0.822909
40886   -1.012800
40887   -1.320214
40888   -1.369441
40889   -1.735148
Name: EquityToImmobilizedAssets, Length: 40885, dtype: float64

In [141]:
df_for_share.to_csv("DataBanksFinal.csv", header = True, index = False)

In [88]:
df_for_share.NetIncome

0        0.007337
1        0.006679
2        0.007374
3        0.002215
4        0.002413
           ...   
40705    0.023281
40706    0.022948
40707    0.016742
40708    0.018166
40709    0.013099
Name: NetIncome, Length: 40705, dtype: float64

# Moodys 

The code below performs feature selection. I do not recommend to reproduce it, since it takes long (about 2 hours). Hard-coded list of features is copypasted from output.

In [237]:
short_feature_list = ['Securities', 'LendCorporate_Me', 
                      'BorrowRetail_Me', 'LendLong_Me',
                      'Cash_Me', 'BondsIssued_Me',
                      'TradingAccounts_Me', 'BadBorrowing_Me', 
                      'ObligatoryReserves_CBR', 'CoreFunds_CBR',
                      'Intangibles_CBR', 'OtherAssets_CBR', 
                      'SecuritiesIssued_CBR', 'DerivativeLiabilities_CBR',
                      'OtherLiabilities_CBR', 'Revaluation_CBR',
                      'ExtraEquity_CBR', 'ReserveFund_CBR', 
                      'SecuritiesRevaluation_CBR', 'form_public']

### Moodys Logistic Regression

In [238]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Moodys_train, Moodys_train)
estimator.score(X_Moodys_test, Moodys_test)

0.11277173913043478

In [239]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Moodys_train[short_feature_list], Moodys_train)
estimator.score(X_Moodys_test[short_feature_list], Moodys_test)

0.5801630434782609

# ExpertRA

In [240]:
X_Expertra.Assets = (X_Expertra.Assets - X_Expertra.Assets.mean())/(X_Expertra.Assets.std(ddof = 0))

Even having been normalized, the selection for expertra rating converges only at very lots of iterations, which takes a lot of time. Further I will use the same short list of most useful variables as for Moodys.

### Expertra Logistic Regression

In [241]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Expertra_train, Expertra_train)
estimator.score(X_Expertra_test, Expertra_test)

0.01057340382269215

In [242]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_Expertra_train[short_feature_list], Expertra_train)
estimator.score(X_Expertra_test[short_feature_list], Expertra_test)

0.3607157381049207

# Defaults

In [243]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn import model_selection
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

In [244]:
path = "C:\\Users\\Ivand\\Repos\\banks\\data"

In [245]:
df = pd.read_csv(path + "\\BanksDFUsable.csv")
df = df.drop(df.columns[0], axis = 1)
df_ratings = pd.read_csv(path + "\\RatingsClean.csv")

In [246]:
df = df[(df.Year >= 2015)&(df.Year <= 2019)]

In [247]:
#count non-zeros in each column
df.astype(bool).sum(axis = 0).sort_values()   

BadBorrowing_Me               1050
Revaluation_CBR               1405
DefaultIn365Days              3397
DerivativeLiabilities_CBR     5085
CBRCredits_CBR                5379
                             ...  
Equity                       35724
REGN                         35726
Year                         35726
Month                        35726
Date                         35726
Length: 62, dtype: int64

In [248]:
df_ratings = df_ratings.rename(columns = {"regn" : "REGN"})

In [249]:
df.REGN = df.REGN.apply(str)
df_ratings.REGN = df_ratings.REGN.apply(str)

In [250]:
df_ratings[df_ratings.REGN == "1"]

,Moodys,name,ExpertRA,short_name,form,REGN,region,ownership,personal_page,app,Year,Month,foreign
31345,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,1,1.0
31346,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,2,1.0
31347,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,3,1.0
31348,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,4,1.0
31349,no,ЮниКредит Банк,no,ЮниКредит Банк,private,1,moscow,private,1,1,2015,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31412,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,8,1.0
31413,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,9,1.0
31414,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,10,1.0
31415,no,ЮниКредит Банк,ruAAA,ЮниКредит Банк,private,1,moscow,private,1,1,2020,11,1.0


In [251]:
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-К", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Д", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-С", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Г", "", x))
df_ratings.REGN = df_ratings.REGN.apply(lambda x: re.sub("-Р", "", x))

df_full = df.merge(df_ratings, on = ["Year", "Month", "REGN"], how = "left")
#этот фильтр удаляет только национальный клиринговый центр
df_full = df_full[df_full.short_name == df_full.short_name]
df_full

,REGN,Year,Month,NetIncome,NetSecurityIncome,NetDerivativeIncome,NetFeeIncome,NetInvestmentIncome,NetInterestIncome,Assets,...,Moodys,name,ExpertRA,short_name,form,region,ownership,personal_page,app,foreign
0,1,2015,1,8872670.0,-711540.0,16159570.0,8903905.0,1.0,28990932.0,1.209364e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
1,1,2015,2,8872670.0,-711540.0,16159570.0,8903905.0,1.0,28990932.0,1.328361e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
2,1,2015,3,8872670.0,-711540.0,16159570.0,8903905.0,1.0,28990932.0,1.203223e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
3,1,2015,4,2586956.0,-194556.0,3375908.0,1570702.0,-0.0,4807043.0,1.167734e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
4,1,2015,5,2586956.0,-194556.0,3375908.0,1570702.0,-0.0,4807043.0,1.072174e+09,...,no,ЮниКредит Банк,no,ЮниКредит Банк,private,moscow,private,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35803,992,2016,8,51261.0,85340.0,66247.0,8122.0,-0.0,17486.0,2.201881e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0
35804,992,2016,9,51261.0,85340.0,66247.0,8122.0,-0.0,17486.0,2.233772e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0
35805,992,2016,10,37506.0,156479.0,84845.0,13325.0,-14.0,-47470.0,2.240207e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0
35806,992,2016,11,37506.0,156479.0,84845.0,13325.0,-14.0,-47470.0,2.064643e+06,...,no,Вологдабанк,no,Вологдабанк,private,notmoscow,private,0.0,0.0,0.0


In [252]:
df_full = df_full[df_full.Assets != 0]
df_full = df_full[df_full.NetIncome != 0]


In [253]:
divide_by_net_income = ["NetSecurityIncome", 'NetDerivativeIncome',
         'NetFeeIncome', 'NetInvestmentIncome',
         'NetInterestIncome']
df_full.loc[:, divide_by_net_income] = df_full.loc[:, divide_by_net_income].div(df_full["NetIncome"], axis=0)

divide_by_assets = ["NetIncome", 'Liabilities', 'Equity', 'Securities',
       'Investment', 'RetainedEarning','Reserves_Me', 'LendCorporate_Me', 'LendRetail',
       'BorrowCorp_Me', 'BorrowRetail_Me', 'LendInterbank_Me',
       'BorrowInterbank_Me', 'LendShort_Me', 'BorrowShort_Me', 'LendLong_Me',
       'BorrowLong_Me', 'Cash_Me', 'BondsIssued_Me', 'BadCredits_Me',
       'LendState_Me', 'BorrowState_Me', 'CurrentAccounts_Me',
       'TradingAccounts_Me', 'BadBorrowing_Me', 'Unclassified',
       'CBRDeposits_CBR', 'ObligatoryReserves_CBR', 'CreditsToBanks_CBR',
       'Securities_CBR', 'ShareCapitalParticipation_CBR',
       'CreditPortfolio_CBR', 'ProfitableDerivatives_CBR', 'CoreFunds_CBR',
       'Intangibles_CBR', 'DeferredTaxAsset_CBR', 'OtherAssets_CBR',
       'CBRCredits_CBR', 'BanksFunds_CBR', 'ClientFunds_CBR',
       'SecuritiesIssued_CBR', 'DerivativeLiabilities_CBR',
       'OtherLiabilities_CBR', 'Revaluation_CBR', 'MainEquity_CBR',
       'ExtraEquity_CBR', 'ReserveFund_CBR', 'SecuritiesRevaluation_CBR',
       'RetainedEarnings_CBR']
df_full.loc[:, divide_by_assets] = df_full.loc[:, divide_by_assets].div(df_full["Assets"], axis=0)

In [254]:
df_full.drop(["Date", "short_name", "name"], axis = 1, inplace = True)

In [255]:
df_full = pd.get_dummies(df_full, columns = ["ownership", "form", "region"], drop_first = True)

In [256]:
seed = 42
X = df_full.drop(['DefaultIn365Days', 'DefaultIn730Days',
       'DefaultIn10000Days'], axis = 1)
X.drop(["Moodys", "ExpertRA"], axis = 1, inplace = True)
OneYear = df_full['DefaultIn365Days']
TwoYears = df_full["DefaultIn730Days"]
AllYears = df_full["DefaultIn10000Days"]

X_train, X_test, OneYear_train, OneYear_test, TwoYears_train, TwoYears_test, AllYears_train, AllYears_test =\
                    model_selection.train_test_split(X, OneYear, TwoYears, AllYears, test_size=0.2, random_state=seed)


X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
OneYear_train.reset_index(inplace=True, drop=True)
OneYear_test.reset_index(inplace=True, drop=True)
TwoYears_train.reset_index(inplace=True, drop=True)
TwoYears_test.reset_index(inplace=True, drop=True)
AllYears_train.reset_index(inplace=True, drop=True)
AllYears_test.reset_index(inplace=True, drop=True)

рое, роа, проценты по кредитам корпоративным (и тд) к активам, спрэд

среднерыночная доходность кредитных портфелей

корректировка пропорционально доходности портфеля банка к доходности кредитного портфеля (перенос кредитов в иммобилизованные активы)

Презентация: сделать ноутбук воспроизводимым; kaggle; графики; рейтинговые группы; 

Сравнить с корректировками и без корректировок

Сравнить со внешними рейтингами

Идея: график, на одной оси внешние рейтинги, на другой внутренние



## OneYear

The code below performs feature selection. I do not recommend to reproduce it, since it takes long (about 2 hours). Hard-coded list of features is copypasted from output.

In [257]:
short_feature_list_defaults = ['Liabilities',
                             'Equity',
                             'Securities',
                             'Investment',
                             'RetainedEarning',
                             'LendCorporate_Me',
                             'LendRetail',
                             'LendInterbank_Me',
                             'BorrowShort_Me',
                             'LendLong_Me',
                             'BondsIssued_Me',
                             'BadCredits_Me',
                             'LendState_Me',
                             'BadBorrowing_Me',
                             'ClientFunds_CBR',
                             'OtherLiabilities_CBR',
                             'Revaluation_CBR',
                             'MainEquity_CBR',
                             'personal_page',
                             'region_notmoscow']

In [258]:
def Gini(y, y_pred):
    res = roc_auc_score(y, y_pred) * 2 - 1
    print(f"Gini: {res}")
    return(res)


In [259]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train, OneYear_train)
print(f"Accuracy is {estimator.score(X_test, OneYear_test)}")
Gini(list(OneYear_test), estimator.predict_proba(X_test)[:,1])

Accuracy is 0.9044621625402154
Gini: 0.16530299949414418


0.16530299949414418

In [260]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train[short_feature_list_defaults], OneYear_train)
print(f"Accuracy is {estimator.score(X_test[short_feature_list_defaults], OneYear_test)}")
Gini(list(OneYear_test), estimator.predict_proba(X_test[short_feature_list_defaults])[:,1])

Accuracy is 0.9163519373338929
Gini: 0.739718831106194


0.739718831106194

In [267]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train[short_feature_list_defaults], OneYear_train)
Gini(list(OneYear_test), classifier.predict_proba(X_test[short_feature_list_defaults])[:,1])

Gini: 0.9507216257672184


0.9507216257672184

## TwoYears

We won't retrain feature selection, because it takes too long

In [261]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train, TwoYears_train)
print(f"Accuracy is {estimator.score(X_test, TwoYears_test)}")
Gini(list(TwoYears_test), estimator.predict_proba(X_test)[:,1])

Accuracy is 0.8345223108127011
Gini: 0.20537851998178436


0.20537851998178436

In [262]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train[short_feature_list_defaults], TwoYears_train)
print(f"Accuracy is {estimator.score(X_test[short_feature_list_defaults], TwoYears_test)}")
Gini(list(TwoYears_test), estimator.predict_proba(X_test[short_feature_list_defaults])[:,1])

Accuracy is 0.8631976500209819
Gini: 0.7486245671087979


0.7486245671087979

In [268]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train[short_feature_list_defaults], TwoYears_train)
Gini(list(TwoYears_test), classifier.predict_proba(X_test[short_feature_list_defaults])[:,1])

Gini: 0.978283675117013


0.978283675117013

## AllYears

We won't retrain feature selection, because it takes too long

In [263]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train, AllYears_train)
print(f"Accuracy is {estimator.score(X_test, AllYears_test)}")
Gini(list(AllYears_test), estimator.predict_proba(X_test)[:,1])

Accuracy is 0.7329696461043502
Gini: 0.30553225180842847


0.30553225180842847

In [264]:
estimator=LogisticRegression(penalty="none", max_iter=1000, n_jobs = 4)
estimator.fit(X_train[short_feature_list_defaults], AllYears_train)
print(f"Accuracy is {estimator.score(X_test[short_feature_list_defaults], AllYears_test)}")
Gini(list(AllYears_test), estimator.predict_proba(X_test[short_feature_list_defaults])[:,1])

Accuracy is 0.8178766261015527
Gini: 0.7021217295334676


0.7021217295334676

In [269]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train[short_feature_list_defaults], AllYears_train)
Gini(list(AllYears_test), classifier.predict_proba(X_test[short_feature_list_defaults])[:,1])

Gini: 0.9890895477029258


0.9890895477029258